In [ ]:
# Install the required packages
%pip install -r requirements.txt

# Visualize SDS Data

The provided code snippet is a Jupyter Notebook cell that allows users to select and visualize data from .sds files using a dropdown. Run the block to populate the dropdown menu with the files in the folder Test.

In [ ]:
# Run this cell at least once, for core SDS functions to be available
import os
import yaml
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from armsdslib import plotData, RecordManager


def read_metadata(file_path):
    """
    Reads the sds.yml metadata file and returns its contents as a dictionary.
    """
    base_name = os.path.basename(file_path).rsplit('.', 2)[0]
    metadata_path = os.path.join(os.path.dirname(file_path), f"{base_name}.sds.yml")
    if os.path.exists(metadata_path):
        with open(metadata_path, 'r') as file:
            metadata = yaml.load(file, Loader=yaml.FullLoader)["sds"]
        print(f"Metadata loaded from {metadata_path}")
        return metadata
    else:
        print(f"Metadata file not found: {metadata_path}")
        return None

def display_file_content(change):
    """
    Callback function to display the content of the selected SDS file.
    This function reads the selected SDS file, processes the data, and visualizes it using plotData from sdslib.
    """
    
    print("Callback function called")
    print(change)
    selected_filename = change['new']
    if selected_filename == "Select SDS File...":
        return
    
    selected_filepath = sds_files[sds_filenames.index(selected_filename) - 1]
    print(f"Selected file: {selected_filepath}")
    
    # Read metadata
    metadata = read_metadata(selected_filepath)
    if metadata is None:
        return
    
    print(f"Metadata: {metadata}")
    # Create RecordManager instance
    record_mgr = RecordManager()
    print("RecordManager instance created")

    # Open and read the SDS file
    with open(selected_filepath, 'rb') as file:
        data = record_mgr.getData(file)
    print("SDS file data read")

    # Check if data is correctly parsed
    if not isinstance(data, bytearray):
        print(f"Error: Unable to parse data from {selected_filename}")
        return

    # Extract metadata information
    data_name = metadata["name"]
    data_desc = metadata["content"]
    data_freq = metadata["frequency"]
    if not data_freq > 0:
        print(f"Error: Sample frequency must be greater than 0 (f = {data_freq})")
        return

    # Plot data using plotData from sdslib
    plotData(data, data_desc, data_freq, data_name, view3D=False)

In [ ]:
# Run this cell to scan for new SDS files in subfolders of the sds-examples directory
# This cell will create a dropdown widget for selecting the SDS
sds_files = []
sds_filenames = ["Select SDS File..."]

# Walk through the directory to find .sds files
for root, dirs, files in os.walk('..'):
    for file in files:
        if file.endswith('.sds'):
            relative_path = os.path.relpath(os.path.join(root, file), start='..')
            sds_files.append(os.path.join(root, file))
            sds_filenames.append(relative_path)

# Create a dropdown widget for selecting SDS files
dropdown = widgets.Dropdown(
    options=sds_filenames,
    description='SDS Files:',
    disabled=False,
)


# Attach the callback function to the dropdown
dropdown.observe(display_file_content, names='value')

# Display the dropdown widget
display(dropdown)

